<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class3-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 3.2**

**Announcements**
* No class on Monday (University Holiday, *Indigenous Peoples' Day*)
* Lab 3: Available later today
  * Still due Mon 10/10, anytime
* For next class (Wed 10/12), watch Video No. 6: *Digital Low-pass Filters*





# Optional stuff

## Interactive Matplotlib

Install `ipympl` for  Matplotlib


In [ ]:
#!pip install ipympl   # Also installs a more recent version of matplotlib (v3.5.3)

Enable interactive Matplotlib figures

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()
# %matplotlib widget

## My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Setup
As always, start by importing the "usual" modules we'll be using...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal

rc('animation', html='jshtml')

path = '/content/drive/My Drive/eces435-work/class3.2/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'dx21'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Helper functions

## `myPlot()` *Updated*
A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)
  * *New* `x_lim=#` or `x_lim=[x1,x2]` Specify the x-axis limit(s) of the plot
  * *New* `y_lim=#` or `y_lim=[y1,y2]` Specificy the y-axis limit(s) of the plot

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt='', x_lim=0, y_lim=0):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( t[:N], sig[:N], fmt, linewidth=lw)

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = N/fs   # End of input signal
    else:
      x_lim2 = x_lim
    plt.xlim(0, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim != 0:
      plt.ylim(-y_lim, y_lim)
  else:
    plt.ylim(y_lim)

  fig.tight_layout()
  #plt.ion()
  plt.close()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

## `myPlotFFT()`
Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of window to use (default: length of input)
  * `win='hann'` The type of window to use (default: `hann`, or `rect`)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, win='hann', neg_f=False, x_lim=0, fig_size=(16,6),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)

  if win=='hann':  
    win = np.hanning(n_win)
    win_scale = 2
  else:
    win = np.ones(n_win)
    win_scale = 1

  S = np.fft.fft(sig * win, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  if neg_f:
    f = f - (fs/2)
    S = np.fft.fftshift(S)

  S_mag = 2*win_scale*np.abs(S) / n_win     # Frequency magnitude, normalized by length
                                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                            #    x2 for Hann because window has 0.5 average

  S_mag += 1e-15                  # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w


  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = fs/2
    if neg_f:
      x_lim = -fs/2
    plt.xlim(x_lim, x_lim2)
  else:
    plt.xlim(x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

# Load this week's audio sample
Yo Yo Ma: Bach Cello Suite No. 1
* 44.1 kHz sampling rate

In [ ]:
yy44,fs44 = sf.read(path + 'YoYoMa-44kHz.wav')
ipd.Audio(yy44,rate=fs44)

In [ ]:
myPlot(yy44)

# Quantization

In [ ]:
yy44_q = np.floor(yy44 * 512)

myPlot(yy44_q, x_lim=[1,1.1], y_lim=[-512,512])
ipd.Audio(yy44_q, rate=fs44)

In [ ]:
q_noise = yy44*512 - yy44_q 
myPlot(q_noise, x_lim=[1,1.1], y_lim=[-1, 2])
ipd.Audio(q_noise, rate=fs44)

More examples of different quantization levels, *now with normalization*

In [ ]:
yy_q16 = np.floor(yy44 * 32768) / 32768
yy_q8 = np.floor(yy44 * 512) / 512
yy_q4 = np.floor(yy44 * 8) / 8

In [ ]:
myPlot(yy_q4, x_lim=[1,1.1])

In [ ]:
ipd.Audio(yy_q4, rate=fs44)

## Write a quantization function

### `quantize()`

Quantize an input signal to a specified precision per sample (in bits).
* Required arguments:
  * `sig` Input signal (first argument)
  * `n_bits` The number of bits per sample
* Optional arguments:
  * `norm=True/False` Normalize the quantized signal and noise outputs to be within $\pm1$ (default: `True`)
* Outputs:
  * `sig_q` The quantized signal
  * `q_noise` The quantization noise

In [ ]:
def quantize(sig, n_bits, norm=True):
  q_scale = 

  sig_q = 
  q_noise = 

  if norm:
    sig_q = sig_q / 
    q_noise = q_noise / 

  return sig_q, q_noise

### Test your `quantize()` function

In [ ]:
n_bits = 8    # Try different numbers

yy_q, q_noise = quantize(yy44, n_bits, norm=False)
ipd.Audio(yy_q, rate=fs44)

In [ ]:
fig1 = myPlot(yy_q, x_lim=[5,5.1], y_lim=[ ]) # Adjust y-axis limits
fig1

When you're ready, save your image to the class Google Drive folder

In [ ]:
filename = 'sig_q' + str(n_bits) + '.png'
fig1.savefig(path + username + filename)

In [ ]:
myPlot(q_noise, x_lim=[5,5.1], y_lim=[-0.5,1.5])

## Signal to (Quantization) Noise Ratio

In [ ]:
num_bits = 4    # Try different numbers

yy_q2, q_noise2 = quantize(yy44, num_bits, norm=True)

sig_pow = np.sum(yy44**2) / len(yy44)
noise_pow = np.sum(q_noise2**2) / len(q_noise2)

sqnr = sig_pow / noise_pow
print('SQNR:', sqnr)

Do we really need to divide by the number of samples?

In [ ]:
sig_pow2 = np.sum(yy44**2)
noise_pow2 = np.sum(q_noise2**2)

sqnr = sig_pow2 / noise_pow2
print('SQNR 2:', sqnr)

In [ ]:
sqnr_dB = 20 * np.log10(sqnr)
print('SQNR (dB):', sqnr_dB)

Remember, SNR is a *ratio*
* $\log\left(\frac{a}{b}\right) = \log(a) - \log(b)$

Rewrite SNR as a *subtraction*.

In [ ]:
sqnr_dB2 = 
print('SQNR 2 (dB):', sqnr_dB2)

So, how much SNR do we gain, *per each bit* of quantization?

In [ ]:
sqnr_per_bit_dB = 
print('SQNR per bit (dB):', sqnr_per_bit_dB)

## What does quantization do in the frequency domain?

In [ ]:
myPlotFFT(yy44)

In [ ]:
myPlotFFT(yy_q2)

### Spectrograms

In [ ]:
f1, t1, Sxx = signal.spectrogram(yy44, fs44, window='hann', nperseg=2048, noverlap=2048-512, nfft=4096)

fig = plt.figure(figsize=(16,6))

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)+1e-15))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

In [ ]:
f1, t1, Sxx = signal.spectrogram(yy_q, fs44, window='hann', nperseg=2048, noverlap=2048-512, nfft=4096)

fig = plt.figure(figsize=(16,6))

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)+1e-15))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

### Time to write a `mySpectrogram()` function
A simple wrapper to compute and plot the spectrogram of a signal with my default figure settings, a time x-axis (in seconds) and a frequency y-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `win='window_name'` The type of analysis window to use (default: 'hann')
  * `n_win=#` The length of window to use per frame (default: 1024)
  * `n_fft=#` The size of FFT to use (default: 1024)
  * `x_lim=# or (#,#)` x-axis limit or range (in seconds)
  * `y_lim=# or (#,#)` y-axis limit or range (in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x-ax=True/False` Show x-axis (default: True)
  * `y-ax=True/False` Show y-axis (default: True)

In [ ]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)

  S_mag = 
  S_dBFS = 
  
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

In [ ]:
mySpectrogram(yy44)

In [ ]:
mySpectrogram(yy_q16)

# Bitrate

What's the bitrate for these various quantizations?
* $b_r = f_s \cdot B \cdot C$ in bits / second
  * $f_s$ is the sampling rate
  * $B$ is bits per sample
  * $C$ is the number of channels

In [ ]:
br44_16 = fs44 * 16
print(br44_16)

## What are other bitrates?

Compute some variations
* Different sampling rates
* Different bit depths (quantization levels)

*And what is a typical streaming bitrate (Spotify, Apple Music, etc.)?*

In [ ]:
my_br = 

# Load low sampling rate version (11025 Hz sampling)

In [ ]:
yy11,fs11 = sf.read(path + 'YoYoMa-11kHz.wav')
ipd.Audio(yy11,rate=fs11)

In [ ]:
mySpectrogram(yy11, fs11, y_lim=[0,22050])

#Sample rate conversion: done poorly


## Upsampling: insert zeros between existing samples

In [ ]:
L = 4
yy11_44u = np.zeros( len(yy11)*L - (L-1) )
yy11_44u[::4] = yy11                # Put our 11 kHz wave every 4th sample

ipd.Audio(yy11_44u,rate=fs44)

In [ ]:
mySpectrogram(yy11_44u)

# Moving average smoothing

In [ ]:
L = 7  # Number of samples to average
#L = 17  # Number of samples to average

yy11_44_ma = np.zeros(len(yy11_44u))            # Create an output array of zeros

for n in range(len(yy11_44u)-L):
  yy11_44_ma[n] = np.sum(yy11_44u[n:n+L]) / L   # Compute moving average of L samples

In [ ]:
myPlot(yy11_44_ma[:200])

In [ ]:
ipd.Audio(yy11_44_ma, rate=44100)

In [ ]:
mv_avg = np.ones(L)

myPlotFFT(mv_avg, n_fft=2048, win='rect')

In [ ]:
myPlotFFT(yy11_44_ma)

In [ ]:
mySpectrogram(yy11_44_ma)

## Weighted average smoothing

In [ ]:
L = 17  # Number of samples to average
wt_avg = np.hanning(L)

yy11_44_wt = np.zeros(len(yy11_44u))                  # Create an output array of zeros

for n in range(len(yy11_44u)-L):
  yy11_44_wt[n] = np.sum( yy11_44u[n:n+L] * wt_avg )  # Compute weighted average of L samples

In [ ]:
myPlot(yy11_44_wt[:200])

In [ ]:
ipd.Audio(yy11_44_wt, rate=44100)

In [ ]:
myPlotFFT(wt_avg, n_fft=2048, win='rect')

In [ ]:
mySpectrogram(yy11_44_wt)

In [ ]:
myPlotFFT(yy11_44_wt, win='rect')